In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing.pool import Pool
import re
positive_file = 'data/rt-polarity.pos'
negative_file = 'data/rt-polarity.neg'
data_root = 'data/stanfordSentimentTreebank/'
glove_pattern = 'data/glove.6B.<size>d.txt'
glove_size = 50

In [2]:
def get_glove(size = 50):
    file = re.sub('<size>', str(size), glove_pattern)
    glove = pd.read_csv(file, sep = " ", header = None, index_col = 0, quoting = 3)
    glove = {key: val.values for key, val in glove.T.items()}
    return glove

def preprocess_sentence(line):
    line = re.sub(r'[^\x00-\x7F]+', '', line.strip())
    return line.strip().lower()

def preprocess_for_labels(line):
    line = preprocess_sentence(line)
    line = re.sub('\W+', '', line.strip())
    return line
    
def get_labelset(file):
    with open(file, "rt", encoding="utf-8") as f:
        sentences = set([preprocess_for_labels(line) for line in f.readlines()])
    return sentences

In [3]:
positive_labelset = get_labelset(positive_file)
negative_labelset = get_labelset(negative_file)

In [4]:
data_split_dict = {}
data_split_map = {'1': 'train', '2': 'test', '3':'val'}
with open(data_root + 'datasetSplit.txt', 'r') as f:
    for line in f.readlines():
        [index, set_code] = line.strip().split(',')
        try:
            index = int(index)
        except:
            continue
        if set_code not in data_split_map:
            print(index, set_code)
        data_split_dict[index] = data_split_map.get(set_code, 'val')

In [5]:
def check_sentiment(sentence):
    s = preprocess_for_labels(sentence)
    for y, labelset in enumerate([negative_labelset, positive_labelset]):
        for entry in labelset:
            if s in labelset:
                return y
    return -1

sentences = {}
word2ind = {}
ttws = tf.keras.preprocessing.text.text_to_word_sequence
n_skipped = 0
idx = 0
with open(data_root + 'datasetSentences.txt') as data:
    lines = data.readlines()
    for line in lines:
        index = re.search('^\d+', line)
        if index is None:
            continue
        index = int(index.group())
        if index not in data_split_dict:
            print(index, line)
        entry = {'split_set': data_split_dict.get(index, 'val')}
        line = preprocess_sentence(line)
        line = re.sub(r'^\d+\s+', '', line)
        sentiment = check_sentiment(line)
        if sentiment >= 0:
            entry['y'] = sentiment
        else:
            n_skipped += 1
            continue
        sentences[line] = entry
        tokens = ttws(line)
        for token in tokens:
            if token not in word2ind:
                word2ind[token] = idx
                idx = idx + 1
            
print(n_skipped, ' skipped')
print(len(sentences), ' kept')

2775  skipped
9077  kept


In [6]:
data_splits = {title: {'x':[], 'y':[]} for title in ['train', 'test', 'val']}
for line, entry in sentences.items():
    which = entry['split_set']
    tokenized = [int(word2ind[token]) for token in ttws(line)]
    (data_splits[which]['x']).append(tokenized)
    (data_splits[which]['y']).append(int(entry['y']))

In [27]:
def get_dataset(label, sequence_len):
    vals = data_splits[label]
    x = [np.array(vx) for vx in vals['x']]
    x = tf.keras.preprocessing.sequence.pad_sequences(x, 
                                                      maxlen = sequence_len)
    y = np.array(vals['y'])
    return x, y

In [28]:
def get_embedding_matrix(glove_size):
    glove = get_glove(glove_size)
    glove_words = set(glove.keys())
    default_vector = np.mean(list(glove.values()), axis = 0)
    embedding_matrix = np.empty((len(word2ind), default_vector.shape[0]))
    excluded = []
    for word, position in word2ind.items():
        if word not in glove:
            excluded.append(word)
        embedding_matrix[position,:] = glove.get(word, default_vector)
    print(len(excluded), ' words from dataset not in glove')
    return embedding_matrix

def gen_model(layer_type = tf.keras.layers.SimpleRNN, 
              trainable = False, n_layers = 1, 
              n_hidden_states = 8, lr = .01,
              embedding_matrix = None, glove_size = 300):
    if embedding_matrix is None:
        embedding_matrix = get_embedding_matrix(glove_size)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], 
                               embedding_matrix.shape[1], 
                               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                               trainable = trainable)
             )
    for n in range(n_layers - 1):
        model.add(layer_type(n_hidden_states, return_sequences = True))
    model.add(layer_type(n_hidden_states))
    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['BinaryAccuracy'])
    return model

In [33]:
def run_model(epochs = 100, batch_size = 500, sequence_len = 50, model_args = {}):
    (xtrain, ytrain)= get_dataset('train', sequence_len)
    (xtest, ytest) = get_dataset('test', sequence_len)
    (xval, yval) = get_dataset('val', sequence_len)
    model = gen_model(**model_args)
    train_history = model.fit(xtrain, ytrain, 
                              validation_data = (xval, yval),
                              batch_size = batch_size,
                              epochs = epochs)
    return train_history, model

In [34]:
vanilla_rnn_hist = run_model()

398  words from dataset not in glove
Train on 6530 samples, validate on 845 samples
Epoch 1/100
6530/6530 [==============================] - 1s 179us/sample - loss: 0.7023 - binary_accuracy: 0.5613 - val_loss: 0.6529 - val_binary_accuracy: 0.6178
Epoch 2/100
6530/6530 [==============================] - 0s 42us/sample - loss: 0.6304 - binary_accuracy: 0.6346 - val_loss: 0.6469 - val_binary_accuracy: 0.6260
Epoch 3/100
6530/6530 [==============================] - 0s 44us/sample - loss: 0.6121 - binary_accuracy: 0.6590 - val_loss: 0.6194 - val_binary_accuracy: 0.6686
Epoch 4/100
6530/6530 [==============================] - 0s 44us/sample - loss: 0.5703 - binary_accuracy: 0.7034 - val_loss: 0.6277 - val_binary_accuracy: 0.6651
Epoch 5/100
6530/6530 [==============================] - 0s 43us/sample - loss: 0.5350 - binary_accuracy: 0.7386 - val_loss: 0.5186 - val_binary_accuracy: 0.7527
Epoch 6/100
6530/6530 [==============================] - 0s 44us/sample - loss: 0.5304 - binary_accuracy:

Epoch 51/100
6530/6530 [==============================] - 0s 42us/sample - loss: 0.4105 - binary_accuracy: 0.8165 - val_loss: 0.6002 - val_binary_accuracy: 0.7385
Epoch 52/100
6530/6530 [==============================] - 0s 43us/sample - loss: 0.5389 - binary_accuracy: 0.7521 - val_loss: 0.6064 - val_binary_accuracy: 0.6888
Epoch 53/100
6530/6530 [==============================] - 0s 43us/sample - loss: 0.4774 - binary_accuracy: 0.7758 - val_loss: 0.5895 - val_binary_accuracy: 0.7148
Epoch 54/100
6530/6530 [==============================] - 0s 41us/sample - loss: 0.4515 - binary_accuracy: 0.7959 - val_loss: 0.5682 - val_binary_accuracy: 0.7396
Epoch 55/100
6530/6530 [==============================] - 0s 43us/sample - loss: 0.4178 - binary_accuracy: 0.8168 - val_loss: 0.5649 - val_binary_accuracy: 0.7408
Epoch 56/100
6530/6530 [==============================] - 0s 42us/sample - loss: 0.4214 - binary_accuracy: 0.8107 - val_loss: 0.5551 - val_binary_accuracy: 0.7456
Epoch 57/100
6530/6530

In [35]:
vanilla_lstm_hist = run_model(model_args = {'layer_type': tf.keras.layers.LSTM})

398  words from dataset not in glove
Train on 6530 samples, validate on 845 samples
Epoch 1/100
6530/6530 [==============================] - 2s 308us/sample - loss: 0.6251 - binary_accuracy: 0.6510 - val_loss: 0.4907 - val_binary_accuracy: 0.7799
Epoch 2/100
6530/6530 [==============================] - 1s 127us/sample - loss: 0.4956 - binary_accuracy: 0.7632 - val_loss: 0.4784 - val_binary_accuracy: 0.7858
Epoch 3/100
6530/6530 [==============================] - 1s 100us/sample - loss: 0.4577 - binary_accuracy: 0.7870 - val_loss: 0.4493 - val_binary_accuracy: 0.7905
Epoch 4/100
6530/6530 [==============================] - 1s 102us/sample - loss: 0.4197 - binary_accuracy: 0.8100 - val_loss: 0.4454 - val_binary_accuracy: 0.7882
Epoch 5/100
6530/6530 [==============================] - 1s 102us/sample - loss: 0.3952 - binary_accuracy: 0.8194 - val_loss: 0.4510 - val_binary_accuracy: 0.7870
Epoch 6/100
6530/6530 [==============================] - 1s 103us/sample - loss: 0.3864 - binary_accu

6530/6530 [==============================] - 1s 102us/sample - loss: 0.0325 - binary_accuracy: 0.9942 - val_loss: 1.1193 - val_binary_accuracy: 0.7680
Epoch 51/100
6530/6530 [==============================] - 1s 104us/sample - loss: 0.0293 - binary_accuracy: 0.9949 - val_loss: 1.2222 - val_binary_accuracy: 0.7420
Epoch 52/100
6530/6530 [==============================] - 1s 99us/sample - loss: 0.0531 - binary_accuracy: 0.9864 - val_loss: 1.0458 - val_binary_accuracy: 0.7609
Epoch 53/100
6530/6530 [==============================] - 1s 99us/sample - loss: 0.0778 - binary_accuracy: 0.9753 - val_loss: 1.1720 - val_binary_accuracy: 0.7598
Epoch 54/100
6530/6530 [==============================] - 1s 99us/sample - loss: 0.0582 - binary_accuracy: 0.9856 - val_loss: 1.0643 - val_binary_accuracy: 0.7574
Epoch 55/100
6530/6530 [==============================] - 1s 98us/sample - loss: 0.0404 - binary_accuracy: 0.9908 - val_loss: 1.2079 - val_binary_accuracy: 0.7538
Epoch 56/100
6530/6530 [=========

6530/6530 [==============================] - 1s 103us/sample - loss: 0.0085 - binary_accuracy: 0.9998 - val_loss: 1.4365 - val_binary_accuracy: 0.7621


In [36]:
vanilla_gru_hist = run_model(model_args = {'layer_type': tf.keras.layers.GRU})

398  words from dataset not in glove
Train on 6530 samples, validate on 845 samples
Epoch 1/100
6530/6530 [==============================] - 2s 277us/sample - loss: 0.6417 - binary_accuracy: 0.6250 - val_loss: 0.5556 - val_binary_accuracy: 0.7266
Epoch 2/100
6530/6530 [==============================] - 1s 93us/sample - loss: 0.5248 - binary_accuracy: 0.7380 - val_loss: 0.4822 - val_binary_accuracy: 0.7728
Epoch 3/100
6530/6530 [==============================] - 1s 93us/sample - loss: 0.4636 - binary_accuracy: 0.7838 - val_loss: 0.4630 - val_binary_accuracy: 0.7882
Epoch 4/100
6530/6530 [==============================] - 1s 94us/sample - loss: 0.4305 - binary_accuracy: 0.8034 - val_loss: 0.4639 - val_binary_accuracy: 0.7905
Epoch 5/100
6530/6530 [==============================] - 1s 96us/sample - loss: 0.3971 - binary_accuracy: 0.8179 - val_loss: 0.4660 - val_binary_accuracy: 0.7834
Epoch 6/100
6530/6530 [==============================] - 1s 97us/sample - loss: 0.4008 - binary_accuracy:

Epoch 51/100
6530/6530 [==============================] - 1s 101us/sample - loss: 0.0407 - binary_accuracy: 0.9882 - val_loss: 1.2747 - val_binary_accuracy: 0.7562
Epoch 52/100
6530/6530 [==============================] - 1s 93us/sample - loss: 0.0340 - binary_accuracy: 0.9916 - val_loss: 1.2780 - val_binary_accuracy: 0.7527
Epoch 53/100
6530/6530 [==============================] - 1s 95us/sample - loss: 0.0421 - binary_accuracy: 0.9885 - val_loss: 1.2560 - val_binary_accuracy: 0.7621
Epoch 54/100
6530/6530 [==============================] - 1s 124us/sample - loss: 0.0487 - binary_accuracy: 0.9827 - val_loss: 1.3415 - val_binary_accuracy: 0.7479
Epoch 55/100
6530/6530 [==============================] - 1s 146us/sample - loss: 0.0415 - binary_accuracy: 0.9881 - val_loss: 1.4530 - val_binary_accuracy: 0.7503
Epoch 56/100
6530/6530 [==============================] - 1s 108us/sample - loss: 0.0372 - binary_accuracy: 0.9907 - val_loss: 1.3891 - val_binary_accuracy: 0.7550
Epoch 57/100
6530/